In [ ]:
%matplotlib widget

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider
from IPython.display import display

In [ ]:
# Boltzmann constant in J/K
k_B = 1.380649e-23

# Maxwell-Boltzmann velocity distribution
def maxwell_boltzmann(v, T, m):
    """
    Maxwell-Boltzmann velocity distribution
    v: velocity (m/s)
    T: temperature (K)
    m: molecular mass (kg)
    """
    prefactor = 4 * np.pi * (m / (2 * np.pi * k_B * T))**(3/2)
    return prefactor * v**2 * np.exp(-m * v**2 / (2 * k_B * T))

# Most probable velocity
def v_prob(T, m):
    return np.sqrt(2 * k_B * T / m)

# Mean velocity
def v_mean(T, m):
    return np.sqrt(8 * k_B * T / (np.pi * m))

# RMS velocity
def v_rms(T, m):
    return np.sqrt(3 * k_B * T / m)

# Velocity grid
v_max = 2000  # m/s
v = np.linspace(0, v_max, 1000)

# Figure
fig, ax = plt.subplots(figsize=(12, 6))

# Initialize distribution line
line_dist, = ax.plot(v, maxwell_boltzmann(v, 300, 28.014e-3/6.022e23), 'b-', linewidth=2, label='Distribution')

# Vertical lines for characteristic velocities
line_prob, = ax.plot([], [], 'r--', linewidth=1.5, alpha=0.7, label='Most probable')
line_mean, = ax.plot([], [], 'g--', linewidth=1.5, alpha=0.7, label='Mean')
line_rms, = ax.plot([], [], 'purple', linestyle='--', linewidth=1.5, alpha=0.7, label='RMS')

ax.set_xlabel('Velocity (m/s)', fontsize=12)
ax.set_ylabel('Probability Density', fontsize=12)
ax.set_title('Maxwell-Boltzmann Velocity Distribution', fontsize=14)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

# Information box
info_text = ax.text(
    0.98, 0.97, "",
    transform=ax.transAxes,
    fontsize=10,
    verticalalignment='top',
    horizontalalignment='right',
    bbox=dict(boxstyle="round", facecolor="white", alpha=0.9)
)

plt.tight_layout()
plt.show()

# Molecular masses (kg/mol converted to kg/particle)
# Common gases
masses_dict = {
    'H₂ (Hydrogen)': 2.016e-3 / 6.022e23,
    'N₂ (Nitrogen)': 28.014e-3 / 6.022e23,
    'O₂ (Oxygen)': 31.999e-3 / 6.022e23,
    'CO₂': 44.01e-3 / 6.022e23,
    'Ar (Argon)': 39.948e-3 / 6.022e23,
}

# Default mass (N2)
default_mass = masses_dict['N₂ (Nitrogen)']

# Update function
def update(T=300, mass_select='N₂ (Nitrogen)'):
    m = masses_dict[mass_select]
    
    # Update distribution
    dist = maxwell_boltzmann(v, T, m)
    line_dist.set_ydata(dist)
    
    # Calculate characteristic velocities
    v_p = v_prob(T, m)
    v_mn = v_mean(T, m)
    v_r = v_rms(T, m)
    
    # Get max probability for scaling
    max_prob = np.max(dist)
    
    # Update vertical lines
    line_prob.set_data([v_p, v_p], [0, max_prob])
    line_mean.set_data([v_mn, v_mn], [0, max_prob])
    line_rms.set_data([v_r, v_r], [0, max_prob])
    
    # Update info box
    info_text.set_text(
        f"T = {T:.1f} K\n"
        f"Species: {mass_select}\n\n"
        f"v_prob = {v_p:.1f} m/s\n"
        f"v_mean = {v_mn:.1f} m/s\n"
        f"v_rms = {v_r:.1f} m/s"
    )
    
    ax.relim()
    ax.autoscale_view()
    fig.canvas.draw_idle()

# Interactive plot with sliders and dropdown
from ipywidgets import Dropdown
interact(
    update,
    T=FloatSlider(min=100, max=1000, step=50, value=300),
    mass_select=Dropdown(
        options=list(masses_dict.keys()),
        value='N₂ (Nitrogen)',
        description='Gas:'
    )
)